In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession

import os
os.getcwd()
data_folder = '/home/ubuntu/data/raw'
os.chdir(data_folder)
os.getcwd()

'/home/ubuntu/data/raw'

In [8]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [10]:
file = '/Ecommerce_Customers.csv'
data = spark.read.csv(data_folder+file, inferSchema=True, header=True)

In [11]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [20]:
for item in data.head(2)[1]:
    print(item)
data.columns

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [15]:
#Setup DataFrame for Machine Learning
#1. Import Vector assemblers
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [21]:
#2. Grab numerical columns as fuatures and YearlySmount Spent as label
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                      'Length of Membership'],
                           outputCol='features')

In [24]:
output = assembler.transform(data)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [26]:
final_data = output.select('features','Yearly Amount Spent')

In [27]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [29]:
train_data,test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                361|
|   mean|  496.8661917268616|
| stddev|  80.01438244660726|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
#Create LR Model
lr = LinearRegression(labelCol='Yearly Amount Spent', featuresCol='features')

In [31]:
#Train Model
lr_model = lr.fit(train_data)

In [34]:
#Evaluate
test_results = lr_model.evaluate(test_data)
test_results.residuals.show(5)

+------------------+
|         residuals|
+------------------+
|-17.76148330642593|
|-4.369958632090345|
|-6.412468781680275|
|-13.42749616921327|
|22.070056107477285|
+------------------+
only showing top 5 rows



In [36]:
#RMS = (real - predicted)^2
print(test_results.rootMeanSquaredError)
print(test_results.r2)
final_data.describe().show()

#Our RMS is $9.7 - but this is pretty good, because mean is ~500
#Our R2 value is really high too, which means that we explain 98% of the variance in the data

9.753178337021032
0.9840023819393018
+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [38]:
#Deploy on unlabled
unlabled_data = test_data.select('features')
predictions = lr_model.transform(unlabled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8162006488763...|283.84782425489493|
|[30.8364326747734...|471.87185905907995|
|[31.0613251567161...| 493.9679268395819|
|[31.0662181616375...| 462.3607893768876|
|[31.2834474760581...| 569.7110333181902|
|[31.3123495994443...|445.36011581741127|
|[31.4252268808548...| 534.7995965207581|
|[31.5147378578019...|495.66171172162285|
|[31.5257524169682...|450.07553333827514|
|[31.5316044825729...|433.10968717585047|
|[31.5761319713222...| 543.3625172470652|
|[31.8209982016720...|417.91309411427187|
|[31.8293464559211...|384.80047686665625|
|[31.8512531286083...|464.93940730679446|
|[31.8627411090001...| 557.9625040981311|
|[31.8745516945853...|398.91676031240786|
|[31.9120759292006...| 389.9035446074188|
|[31.9365486184489...| 441.4460767700002|
|[31.9673209478824...|450.76661228209036|
|[32.0047530203648...|464.11356868725466|
+--------------------+------------